Endpoint base para listagem, ou método GET

In [2]:
from flask import Blueprint, jsonify, request
from typing import Type

class BaseRoute:
    def __init__(self, name: str, url_prefix: str, service_class: Type):
        self.blueprint = Blueprint(name, __name__, url_prefix=url_prefix)
        self.service_class = service_class

        self.blueprint.add_url_rule("/", view_func=self.listar, methods=["GET"])  

def listar(self):
    session, service = self.get_service()
    try:
        entidades = service.listar()
        return jsonify([e.to_dict() for e in entidades])
    finally:
        session.close()

Endpoint de listagem de consulta, método GET. Lista todas as consultas cadastradas.

In [3]:
from services.consulta_service import ConsultaService
from flask import Blueprint, jsonify, request

consulta_bp = Blueprint("Consulta", __name__, url_prefix="/consultas")

@consulta_bp.route("/", methods=["GET"])
def listar_consultas():
    return jsonify(ConsultaService.listar())

Endpoint para as especialidades. Herda as funções listar, buscar_por_id, criar e deletar da base_route (Métodos GET, POST e DELETE )

In [3]:
from routes.base_route import BaseRoute
from services.especialidade_service import EspecialidadeService

especialidade_route = BaseRoute("Especialidade", "/especialidades", EspecialidadeService)
especialidade_bp = especialidade_route.blueprint

Endpoint para os medicamentos. Herda as funções listar, buscar_por_id, criar e deletar da base_route (Métodos GET, POST e DELETE )

In [4]:
from routes.base_route import BaseRoute
from services.medicamento_service import MedicamentoService

medicamento_route = BaseRoute("Medicamento", "/medicamentos", MedicamentoService)
medicamento_bp = medicamento_route.blueprint

Endpoint para os médicos. Herda as funções listar, buscar_por_id, criar e deletar da base_route (Métodos GET, POST e DELETE )

In [5]:
from routes.base_route import BaseRoute
from services.medico_service import MedicoService

medico_route = BaseRoute("Medico", "/medicos", MedicoService)
medico_bp = medico_route.blueprint

Endpoint para os pacientes. Herda as funções listar, criar e deletar da base_route (Métodos GET, POST e DELETE )

In [6]:
from routes.base_route import BaseRoute
from services.paciente_service import PacienteService

paciente_route = BaseRoute("Paciente", "/pacientes", PacienteService)
paciente_bp = paciente_route.blueprint

Endpoint para listagem das posologias, método GET. Lista todas as posologias registradas.

In [ ]:
from flask import Blueprint, jsonify, request
from services.posologia_service import PosologiaService

posologia_bp = Blueprint("Posologia", __name__, url_prefix="/posologias")

@posologia_bp.route("/", methods=["GET"])
def listar_posologias():
    session = SessionLocal()
    service = PosologiaService(session)
    posologias = service.listar()
    session.close()
    return jsonify([p.to_dict() for p in posologias])

Endpoint para os tratamentos. Herda as funções listar, buscar_por_id, criar e deletar da base_route (Métodos GET, POST e DELETE )

In [8]:
from routes.base_route import BaseRoute
from services.tratamento_service import TratamentoService

tratamento_route = BaseRoute("Tratamento", "/tratamentos", TratamentoService)
tratamento_bp = tratamento_route.blueprint

Endpoint base para o método GET por ID (busca as entidades pelo ID delas)

In [26]:
from flask import Blueprint, jsonify, request
from typing import Type

class BaseRoute:
    def __init__(self, name: str, url_prefix: str, service_class: Type):
        self.blueprint = Blueprint(name, __name__, url_prefix=url_prefix)
        self.service_class = service_class

        self.blueprint.add_url_rule("/<int:entity_id>", view_func=self.buscar_por_id, methods=["GET"])

def buscar_por_id(self, entity_id: int):
        session, service = self.get_service()
        try:
            entidade = service.buscar_por_id(entity_id)
            if entidade:
                return jsonify(entidade.to_dict())
            return jsonify({"erro": "Não encontrado"}), 404
        finally:
            session.close()

Endpoint de listagem de consultas por ID do paciente (Método GET)

In [ ]:
from flask import Blueprint, jsonify, request
from services.consulta_service import ConsultaService

consulta_bp = Blueprint("Consulta", __name__, url_prefix="/consultas")

@consulta_bp.route("/paciente/<int:paciente_id>", methods=["GET"])
def listar_consultas_por_paciente(paciente_id):
    return jsonify(ConsultaService.listar_por_paciente(paciente_id))

Endpoint de listagem de posologias pelo ID do paciente (Método GET)

In [11]:
from flask import Blueprint, jsonify, request
from services.posologia_service import PosologiaService

@posologia_bp.route("/<int:entity_id>", methods=["GET"])
def buscar_posologia(entity_id):
    session = SessionLocal()
    service = PosologiaService(session)
    posologia = service.buscar_por_id(entity_id)
    session.close()
    if posologia:
        return jsonify(posologia.to_dict())
    return jsonify({"erro": "Posologia não encontrada"}), 404

Endpoint base para o método POST

In [27]:
from flask import Blueprint, jsonify, request
from typing import Type

class BaseRoute:
    def __init__(self, name: str, url_prefix: str, service_class: Type):
        self.blueprint = Blueprint(name, __name__, url_prefix=url_prefix)
        self.service_class = service_class

        self.blueprint.add_url_rule("/", view_func=self.criar, methods=["POST"])

def criar(self):
        session, service = self.get_service()
        data = request.get_json()
        try:
            entidade = service.criar(data)
            return jsonify(entidade.to_dict()), 201
        except ValueError as e:
            return jsonify({"erro": str(e)}), 400
        finally:
            session.close()

Endpoint para postagem de uma nova consulta (método POST)

In [ ]:
from flask import Blueprint, jsonify, request
from services.consulta_service import ConsultaService

@consulta_bp.route("/", methods=["POST"])
def criar_consulta():
    data = request.get_json()
    resultado = ConsultaService.criar(data)
    return resultado

Endpoint para postagem de uma nova posologia (Metodo POST)

In [14]:
from flask import Blueprint, jsonify, request
from services.posologia_service import PosologiaService

@posologia_bp.route("/", methods=["POST"])
def criar_posologia():
    session = SessionLocal()
    service = PosologiaService(session)
    data = request.get_json()
    try:
        posologia = service.criar(data)
        session.close()
        return jsonify(posologia.to_dict()), 201
    except ValueError as e:
        session.close()
        return jsonify({"erro": str(e)}), 400

Endpoint base para o método DELETE (Por ID)

In [28]:
from flask import Blueprint, jsonify, request
from typing import Type

class BaseRoute:
    def __init__(self, name: str, url_prefix: str, service_class: Type):
        self.blueprint = Blueprint(name, __name__, url_prefix=url_prefix)
        self.service_class = service_class

        self.blueprint.add_url_rule("/<int:entity_id>", view_func=self.deletar, methods=["DELETE"])

def deletar(self, entity_id: int):
        session, service = self.get_service()
        try:
            entidade = service.deletar(entity_id)
            if entidade:
                return jsonify({"mensagem": "Deletado com sucesso"})
            return jsonify({"erro": "Não encontrado"}), 404
        finally:
            session.close()
    

Endpoint para deleter uma consulta (Método DELETE, por ID)

In [ ]:
from flask import Blueprint, jsonify, request
from services.consulta_service import ConsultaService

@consulta_bp.route("/<int:consulta_id>", methods=["DELETE"])
def deletar_consulta(consulta_id):
    return ConsultaService.deletar(consulta_id)

Endpoint para deleter uma posologia (Método DELETE, por ID)

In [17]:
from flask import Blueprint, jsonify, request
from services.posologia_service import PosologiaService

@posologia_bp.route("/<int:entity_id>", methods=["DELETE"])
def deletar_posologia(entity_id):
    session = SessionLocal()
    service = PosologiaService(session)
    posologia = service.deletar(entity_id)
    session.close()
    if posologia:
        return jsonify({"mensagem": "Posologia deletada"})
    return jsonify({"erro": "Posologia não encontrada"}), 404